#### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/IASNLP"

### Importing Necessary Libraries

In [ ]:
!pip install sentencepiece

In [51]:
import numpy as np
import pandas as pd

import sentencepiece as spm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt 

from functools import reduce
import os

### Loading Data

We have downloaded the entire Samanantar Parallel Corpus. The data is split by source from where it was collected. We are concerned with the English-Bengali Parallel Corpus for the purpose of our training. Hence, we have removed the English to other Indian Language information. We might include them later in our study or training to see if that improves performance.

Due to computational resource constraint, we will take some fraction of data from each source and will pre-process it.

Below, we can see the new created source of parallel corpora from various Indian Channels and Programmes for English to Bengali Data. We also have existing parallel corpora from previous workshops and events included.

In [ ]:
print("New Data created by Samanantar:")
!ls ./Data/source_wise_splits/created
print("\nExisting Data Sources before Samanantar:")
!ls ./Data/source_wise_splits/existing

New Data created by Samanantar:
anuvaad_dw		asianetnews	  ie_news	ocr
anuvaad-general_corpus	coursera	  ie_sports	oneindia
anuvaad_mykhel		dwnews		  ie_tech	pmi
anuvaad_ocr		ie_business	  indiccorp	sentinel
anuvaad_oneindia	ie_education	  khan_academy	wikipedia
anuvaad_pib		ie_entertainment  Kurzgesagt
anuvaad_pib_archives	ie_general	  mykhel
anuvaad_prothomalo	ie_lifestyle	  nptel

Existing Data Sources before Samanantar:
alt	     cvit-pib	   GNOME  Mozilla-I10n	 Tanzil   tico19-terminologies
banglanmt    ELRC_2922	   JW300  OpenSubtitles  Tatoeba  Ubuntu
bible-uedin  GlobalVoices  KDE4   sipc		 TED2020  wikimatrix_opus


In [ ]:
file_names = os.listdir("./Data/source_wise_splits/existing")
data_by_source = list()
for file_name in file_names:
    data_by_source.append(pd.read_csv("./Data/source_wise_splits/existing/"+file_name+"/en-bn/bn_sents.tsv", sep="\t"))

In [ ]:
file_names = os.listdir("./Data/source_wise_splits/created")
for file_name in file_names:
    data_by_source.append(pd.read_csv("./Data/source_wise_splits/created/"+file_name+"/en-bn/bn_sents.tsv", sep="\t"))

In [ ]:
data_by_source = [dat.drop('idx', 1) for dat in data_by_source]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


Let's have a look at one of the loaded files. We can see that there are 3 columns `src` and `tgt` standing for source and target. We now observe that each source sentence has it's bengali translation in the dataset. 

In [ ]:
data_by_source[0].head()

,src,tgt
0,"Okay, I'll be right there.","ওকে, এখনই আসছি"
1,Give one's lessons.,পড়া দেওয়া
2,"Much to the Witnesses' surprise, even the pros...","সাক্ষীরা অত্যন্ত বিস্মিত হন, এমনকি সরকারি উকিল..."
3,I am at your service.,আমি তোমাকে সাহায্য করার জন্যই
4,"Via Facebook Messenger, Global Voices talked w...",গ্লোবাল ভয়েসস বাংলা'র পক্ষ থেকে আমরা যোগাযোগ ক...


Ideally, we will want our training data to be spread across all the sources. Hence, we will merge all the sources together.

In [ ]:
data = pd.concat(data_by_source, ignore_index=True)

In [ ]:
data.to_csv('full_data.csv')

We have in total 9251703 parallel sentences for English to Bengali.

In [5]:
# Run if you have ran the above codes once before and have generated the full_data.csv file
data = pd.read_csv("full_data.csv")
data = data[['src', 'tgt']]

In [6]:
data

,src,tgt
0,"Okay, I'll be right there.","ওকে, এখনই আসছি"
1,Give one's lessons.,পড়া দেওয়া
2,"Much to the Witnesses' surprise, even the pros...","সাক্ষীরা অত্যন্ত বিস্মিত হন, এমনকি সরকারি উকিল..."
3,I am at your service.,আমি তোমাকে সাহায্য করার জন্যই
4,"Via Facebook Messenger, Global Voices talked w...",গ্লোবাল ভয়েসস বাংলা'র পক্ষ থেকে আমরা যোগাযোগ ক...
...,...,...
9251698,A panel of auditors for auditing listed compan...,নিরীক্ষা কার্যক্রমে শৃঙ্খলা আনয়নে তালিকাভুক্ত...
9251699,"On this consideration, and on certain conditio...",সে কারণে কতিপয় শর্ত সাপেক্ষে এসব যন্ত্রাংশের ...
9251700,"Under these rules, fixed price method for IPOs...",এতে অভিহিত মূল্যে আইপিও’র জন্য ফিক্সড প্রাইস প...
9251701,Steps are being taken to introduce rural ratio...,টিআর ও ভিজিএফ এর পরিবর্তে পল্লী রেশনিং কর্মসূচ...


### Data Preparation

We now shuffle the data to form our training data(`train_data`)(which is 1.2% of the entire dataset). We will take some part of this training data(0.05%)(on which the model won't be trained) to form the training-developement(`train_dev_data`) set. The developement set and the test set will be presented later.

Reason: The entire dataset is massive and would take large computing resource to train a model. Hence, we took a subset of the dataset for ease in training.

In [12]:
train_data, train_dev_data  = train_test_split(data, train_size=0.012, test_size=0.0005, random_state=43)

In [5]:
# train_data.to_csv("train_data.csv")
# train_dev_data.to_csv("train_dev.csv")
# Uncomment and run the above lines if running for first time and there is no train_data.csv and train_dev.csv file in the working directory
train_data = pd.read_csv("train_data.csv")[['src', 'tgt']]
train_dev_data = pd.read_csv("train_dev.csv")[['src', 'tgt']]

In [6]:
train_data

,src,tgt
0,But the shoot was a tough one.,তবে শ্যুটটা খুব মুশকিলের ছিল।
1,Road construction started.,রাস্তা নির্মাণ শুরু হয়েছে।
2,Why did he pay so much?,কেন তিনি এত টাকা দিতেন?
3,"""AT ITS worst, this has been Satan's century.","""এই শতাব্দীর প্রচণ্ড ভয়াবহতা এটাকে শয়তানের এক ..."
4,That's our only demand.,সেটাই আমাদের একমাত্র দাবি।
...,...,...
111015,Turmeric powder 1 tea spoon,দারুচিনি-গুঁড়া ১ চা-চামচ।
111016,Photo: Facebook.,ছবি: ফেসবুক থেকে নেয়া।
111017,Strengthening ties with Saudi Arabia,সৌদি আরবের সঙ্গে সম্পর্ক শক্ত করার উদ্যোগ
111018,This is some brief footage Eva took:,এভার তোলা কিছু বিস্তারিত দৃশ্য:


In [7]:
train_dev_data

,src,tgt
0,We beg our Protestant and Jewish friends to pu...,কোন কোন ক্ষেত্রে কর্তৃপক্ষ এবং ধর্মীয় নেতারা ...
1,"Sa'd advised Muhammad: ""Don't be hard on him. ...","সা'দ মুহাম্মাদকে বলেন: ""তার প্রতি কঠোর হবেন না..."
2,Photo by 'Save Gaza Project',"ছবি ""সেভ গাজা প্রজেক্টের""।'"
3,"So, therefore, we need to test batteries under...","অতএব, আমআদের কিছুটা মান অবস্থাগুলির অধীনে ব্যা..."
4,This party is also contesting in the elections.,নির্বাচনে এই দলের মধ্যেই প্রতিদ্বন্দ্বিতা হবে।
...,...,...
4621,But so it is.,তবে তা যেমন ঠিক।
4622,But the work was not started.,তবে কাজ শুরু হয়নি।
4623,The sixth volume of the Polyglot contained var...,পলিগ্লটের ষষ্ঠ খণ্ডে বাইবেল অধ্যয়নের জন্য বিভি...
4624,They need answers.,তাঁদের কাছে জবাব চাওয়া হয়েছে।


Now, one thing we need to decide on is the number of words per sentence that we should take. We look below look at our training data's source sentences to see what kind of input sentences we encounter.

In [20]:
def sent_by_word_count(data, word_counts):
    sentences_filtered_word_num = {}
    for sent in data:
        sent_num_words = len(sent.split())
        for num_words in word_counts:
            if sent_num_words <= num_words:
                sentences_filtered_word_num[num_words] = sentences_filtered_word_num.get(num_words, 0) + 1
    return sentences_filtered_word_num

In [21]:
sentences_filtered_word_num = sent_by_word_count(train_data['src'], list(range(10, 151, 10)))
for num_words, num_sentences in sentences_filtered_word_num.items():
    print(f"The number of english sentences with <= {num_words} words are: {num_sentences}")

The number of english sentences with <= 10 words are: 69625
The number of english sentences with <= 20 words are: 94747
The number of english sentences with <= 30 words are: 105236
The number of english sentences with <= 40 words are: 109011
The number of english sentences with <= 50 words are: 110191
The number of english sentences with <= 60 words are: 110569
The number of english sentences with <= 70 words are: 110761
The number of english sentences with <= 80 words are: 110842
The number of english sentences with <= 90 words are: 110887
The number of english sentences with <= 100 words are: 110925
The number of english sentences with <= 110 words are: 110955
The number of english sentences with <= 120 words are: 110975
The number of english sentences with <= 130 words are: 110989
The number of english sentences with <= 140 words are: 110995
The number of english sentences with <= 150 words are: 110999


In [22]:
sentences_filtered_word_num = sent_by_word_count(train_data['tgt'], list(range(10, 151, 10)))
for num_words, num_sentences in sentences_filtered_word_num.items():
    print(f"The number of bengali sentences with <= {num_words} words are: {num_sentences}")

The number of bengali sentences with <= 10 words are: 76066
The number of bengali sentences with <= 20 words are: 99593
The number of bengali sentences with <= 30 words are: 107265
The number of bengali sentences with <= 40 words are: 109611
The number of bengali sentences with <= 50 words are: 110344
The number of bengali sentences with <= 60 words are: 110647
The number of bengali sentences with <= 70 words are: 110790
The number of bengali sentences with <= 80 words are: 110863
The number of bengali sentences with <= 90 words are: 110920
The number of bengali sentences with <= 100 words are: 110957
The number of bengali sentences with <= 110 words are: 110970
The number of bengali sentences with <= 120 words are: 110982
The number of bengali sentences with <= 130 words are: 110991
The number of bengali sentences with <= 140 words are: 110998
The number of bengali sentences with <= 150 words are: 111004


Looking at the above figures, we see there are not that many sentences of length longer than 60.

Now, we look at the number of unique words in both the source and target sentences.

In [17]:
vocab_en = set([word for sent in train_data['src'].values for word in sent.split()])

In [18]:
vocab_ben = set([word for sent in train_data['tgt'] for word in sent.split()])

In [19]:
print("Number of unique words in english train data:", len(vocab_en))
print("Number of unique words in bengali train data:", len(vocab_ben))

Number of unique words in english train data: 116349
Number of unique words in bengali train data: 148459


### Tokenization and Normalization

We create will create massive text files for both English and Bengali(from the entire dataset except the `train_dev_data`) to train our Byte-Pair Tokenizer for the most frequently occuring characters. 

In [8]:
with open('eng.txt', 'w') as f:
    f.write('\n'.join(train_data['src'].iloc[[idx for idx in train_data.index if idx not in train_dev_data.index]]))

In [9]:
with open('ben.txt', 'w') as f:
    f.write('\n'.join(train_data['tgt'].iloc[[idx for idx in train_data.index if idx not in train_dev_data.index]]))

We train the Byte-Pair Encoder model with vocab size of 16,000 for both English and Bengali.

In [44]:
spm.SentencePieceTrainer.train('--input=eng.txt --model_prefix=eng_bpe --vocab_size=16000 --model_type=bpe --normalization_rule_name=nfkc_cf --bos_id=-1 --eos_id=1 --unk_id=2 --pad_id=0')

In [45]:
spm.SentencePieceTrainer.train('--input=ben.txt --model_prefix=ben_bpe --vocab_size=16000 --model_type=bpe --normalization_rule_name=nfkc_cf --bos_id=-1 --eos_id=1 --unk_id=2 --pad_id=0')

The following two examples show how we the trained BPE encodes subwords to ids, which will be further padded to give rise to the format in which we will input data in our model.

In [46]:
sp_en_bpe = spm.SentencePieceProcessor()
sp_en_bpe.load('eng_bpe.model')

print("*** ENGLISH SENTENCE ***")
print(train_data['src'].iloc[0])
print('*** ENGLISH BPE ***')
print(sp_en_bpe.encode_as_pieces(train_data['src'].iloc[0]))
print(sp_en_bpe.encode_as_ids(train_data['src'].iloc[0]))

*** ENGLISH SENTENCE ***
But the shoot was a tough one.
*** ENGLISH BPE ***
['▁but', '▁the', '▁shoot', '▁was', '▁a', '▁tough', '▁one', '.']
[148, 7, 2619, 86, 5, 5192, 230, 15972]


In [47]:
sp_ben_bpe = spm.SentencePieceProcessor()
sp_ben_bpe.load('ben_bpe.model')

print("*** BENGALI SENTENCE ***")
print(train_data['tgt'].iloc[0])
print('*** BENGALI BPE ***')
print(sp_ben_bpe.encode_as_pieces(train_data['tgt'].iloc[0]))
print(sp_ben_bpe.encode_as_ids(train_data['tgt'].iloc[0]))

*** BENGALI SENTENCE ***
তবে শ্যুটটা খুব মুশকিলের ছিল।
*** BENGALI BPE ***
['▁তবে', '▁শ্যু', 'ট', 'টা', '▁খুব', '▁মু', 'শক', 'িলের', '▁ছিল', '।']
[282, 8825, 15902, 92, 488, 224, 5925, 8969, 155, 15900]


### Build Data Generator

Now, we come to the final phase of data pre-processing i.e. building the data generator which can be feed data to the model continually through the epochs.\
The data generator will yield tuples of English and Bengali sentences where both the English and Bengali sentences are padded and encoded as list of integers corresponding to the respective index in the vocabualary(generated by BPE). 

In [74]:
def data_generator(batch_size, src, tgt, maxlen=60, shuffle=False, verbose=False):
    num_lines = len(src)
    
    lines_index = [*range(num_lines)]
    
    if shuffle:
        np.random.shuffle(lines_index)
    
    index = 0
    while True:
        buffer_src = list()
        buffer_tgt = list() 
                
        max_len = 0 
        for i in range(batch_size):
            if index >= num_lines:
                index = 0
                if shuffle:
                    np.random.shuffle(lines_index)
            
            buffer_src.append(src[lines_index[index]])
            buffer_tgt.append(tgt[lines_index[index]])

            
            
            index += 1


        batch_src = pad_sequences(buffer_src, maxlen = maxlen, padding='post', truncating='post')
        batch_tgt = pad_sequences(buffer_tgt, maxlen = maxlen, padding='post', truncating='post')

        if verbose: print("index=", index)
        yield((batch_src, batch_tgt))

Let's see how our `data_generator` works. But for that we must convert the `src` and `tgt` as list of integers encoded as per BPE vocabulary.

In [55]:
src_train_data_enc = [sp_en_bpe.encode_as_ids(train_data['src'].iloc[i]) for i in range(train_data.shape[0])]
tgt_train_data_enc = [sp_ben_bpe.encode_as_ids(train_data['tgt'].iloc[i]) for i in range(train_data.shape[0])]

In [79]:
dg = data_generator(32, src_train_data_enc, tgt_train_data_enc, verbose=True)

In [80]:
src_batch_1, tgt_batch_1 = next(dg)
src_batch_2, tgt_batch_2 = next(dg)

index= 32
index= 64


In [92]:
print("*** Batch-1 ***")
print("encoded source shape: ", src_batch_1.shape)
print("encoded target shape: ", tgt_batch_1.shape)
print("encoded source example: ", src_batch_1[0])
print("encoded target example: ", tgt_batch_1[0])
print("decoded source example: ", sp_en_bpe.DecodeIdsWithCheck([int(x) for x in list(src_batch_1[0]) if x != 0]))
print("decoded target example: ", sp_ben_bpe.DecodeIdsWithCheck([int(x) for x in list(tgt_batch_1[0]) if x != 0]))

print("\n*** Batch-2 ***")
print("encoded source shape: ", src_batch_2.shape)
print("encoded target shape: ", tgt_batch_2.shape)
print("encoded source example: ", src_batch_2[0])
print("encoded target example: ", tgt_batch_2[0])
print("decoded source example: ", sp_en_bpe.DecodeIdsWithCheck([int(x) for x in list(src_batch_2[0]) if x != 0]))
print("decoded target example: ", sp_ben_bpe.DecodeIdsWithCheck([int(x) for x in list(tgt_batch_2[0]) if x != 0]))

*** Batch-1 ***
encoded source shape:  (32, 60)
encoded target shape:  (32, 60)
encoded source example:  [  148     7  2619    86     5  5192   230 15972     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
encoded target example:  [  282  8825 15902    92   488   224  5925  8969   155 15900     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
decoded source example:  but the shoot was a tough one.
decoded target example:  তবে শ্যুটটা খুব মুশকিলের ছিল।

*** Batch-2 ***
encoded so

We will now use the above data generator to train our models.